## Experimentação de modelos
 Nesta sessão irei realizar uma série de experimentos, combinando diferentes modelos e preprocessamentos. Para registrar tudo isso, irei utilizar o MLFlow.

Aqui irei testar modelos baseados em distância e árvore, segue os esboços dos testes a serem realizados:

Baseados em distância:
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.


Baseados em árvore:
- Diferentes tipos de encoders da biblioteca **category encoders**  para as categóricas
- Diferentes tipos de encoders da biblioteca **category encoders**  para as categóricas + Isolation Forest para rotular os outliers.
Geral:

In [62]:
# Importando as bibliotecas

import mlflow
import pandas as pd

# Pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import IsolationForest

# Modelos
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Metricas
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [50]:
# Importando os dados
dados = pd.read_csv('../data/interim/dados_para_treino.csv', index_col=0)

In [51]:
# Checando a tabela
dados.head()

,category,customer_type,unit_price,quantity,total,payment_type,prcnt_stock,std,var,sem,mean,median,min,max,day_of_week,is_weekend,hour,turn
0,fruit,gold,3.99,2,7.98,e-wallet,0.23,2.161724,4.67305,0.966752,-1.05,-2.24,-2.89,1.67,2,no,9,morning
1,fruit,premium,1.49,1,1.49,credit card,0.54,2.161724,4.67305,0.966752,-1.05,-2.24,-2.89,1.67,2,no,9,morning
2,fruit,non-member,4.49,4,17.96,credit card,0.71,2.161724,4.67305,0.966752,-1.05,-2.24,-2.89,1.67,2,no,9,morning
3,refrigerated items,gold,5.99,1,5.99,credit card,0.54,2.161724,4.67305,0.966752,-1.05,-2.24,-2.89,1.67,2,no,9,morning
4,canned foods,gold,2.49,1,2.49,e-wallet,0.33,2.161724,4.67305,0.966752,-1.05,-2.24,-2.89,1.67,2,no,9,morning


## Buscando o melhor algorítmo

A partir daqui iniciarei os experimentos. Nesta etapa **NÃO** irei alterar os parâmetros, onde o melhor modelo irá receber tunning em outra rodada de experimentos.

Iniciarei os experimentos com os algoritmos **baseados em distância**.

In [32]:
# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

# Dividindo os dados em variáveis dependentes e independentes
x = dados.drop(columns='prcnt_stock')
y = dados.prcnt_stock

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,
                                                        y, test_size=0.25,
                                                        random_state=14)

# Instanciando os modelos
linear_reg = LinearRegression()
lasso = Lasso()
ridge = Ridge()
elastic_nt = ElasticNet()
reg_estocastico = SGDRegressor()

# Criando listas com os modelos e tags
modelos = [linear_reg, lasso, ridge, elastic_nt, reg_estocastico]
tags = ['Reg_Linear', 'Lasso', 'Ridge', 'Elastic_Net', 'Reg_Estocástico']


### Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.

In [63]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes('object').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder', ohe, categoricas)])

In [64]:
pipe = Pipeline([('transformer', transformer),
                 ('regressor', LinearRegression())])

In [65]:
cross_val_score(pipe,
                x_treino,
                y_treino,
                cv=5)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\site-packages\sklearn\linear_model\_base.py", line 648, in fit
    X, y = self._validate_data(
  File "C:\Users\Daniel\OneDrive\Documentos\stock_level_prediction\venv\lib\sit